In [ ]:
def lab_drop_outliner(site):
    dataname = 'lab'
    print('Running p05 '+dataname+' on site '+site, flush = True)                    
    
    datatt = pd.read_pickle('data/'+site+'/p0_'+dataname+'_'+site+'.pkl')
    stats = datatt[datatt['RESULT_NUM'].notnull()][['LAB_LOINC','RESULT_NUM','RESULT_UNIT']].groupby(['LAB_LOINC','RESULT_UNIT']).agg([np.size, np.mean, np.std, np.median, np.var, np.min, np.max, lambda x: np.percentile(x, q=25), lambda x: np.percentile(x, q=75)]).reset_index()
    stats.columns = ['LAB_LOINC','RESULT_UNIT','size','mean','std','median','var','amin','amax','Q1','Q3']
    stats = stats.assign(IQR=lambda x: x.Q3-x.Q1).assign(IQRlwr=lambda x: x['Q1']-1.5*x['IQR']).assign(IQRupr=lambda x: x['Q3']+1.5*x['IQR']).assign(sigmalwr=lambda x: x['mean']-2.7*x['std']).assign(sigmaupr=lambda x: x['mean']+2.7*x['std'])
    datatt = pd.merge(datatt, stats[['LAB_LOINC','RESULT_UNIT', 'IQRlwr', 'IQRupr']], left_on=['LAB_LOINC', 'RESULT_UNIT'], right_on=['LAB_LOINC', 'RESULT_UNIT'], how='left')
    datatt = datatt[datatt['RESULT_NUM'].isnull() | ((datatt['RESULT_NUM'] >= datatt['IQRlwr']) & (datatt['RESULT_NUM'] <= datatt['IQRupr']))].drop(['IQRlwr', 'IQRupr'],axis=1)
    datatt.to_pickle('data/'+site+'/p0_'+dataname+'_'+site+'_drop.pkl')
    
    print('Finished p05 '+dataname+' on site '+site, flush = True)                    

In [ ]:
def vital_drop_outliner(site):
    dataname = 'vital'
    print('Running p05 '+dataname+' on site '+site, flush = True)                    

    datatt = pd.read_pickle('data/'+site+'/p0_'+dataname+'_'+site+'.pkl')

    #stats = datatt['WT'][datatt['WT'].notnull()].agg([np.size, np.mean, np.std, np.median, np.var, np.min, np.max, lambda x: np.percentile(x, q=25), lambda x: np.percentile(x, q=75)]

    datatt2 = pd.melt(datatt, id_vars=[], value_vars=['WT','SYSTOLIC','DIASTOLIC','ORIGINAL_BMI'])

    stats = datatt2[datatt2['value'].notnull()].groupby('variable').agg([np.size, np.mean, np.std, np.median, np.var, np.min, np.max, lambda x: np.percentile(x, q=25), lambda x: np.percentile(x, q=75)])
    stats.columns = ['size','mean','std','median','var','amin','amax','Q1','Q3']
    stats = stats.assign(IQR=lambda x: x.Q3-x.Q1).assign(IQRlwr=lambda x: x['Q1']-1.5*x['IQR']).assign(IQRupr=lambda x: x['Q3']+1.5*x['IQR']).assign(sigmalwr=lambda x: x['mean']-2.7*x['std']).assign(sigmaupr=lambda x: x['mean']+2.7*x['std'])

    datatt['WT'] = datatt['WT'].where(datatt['WT'] >= stats.at['WT','IQRlwr']).where(datatt['WT'] <= stats.at['WT','IQRupr'])
    datatt['ORIGINAL_BMI'] = datatt['ORIGINAL_BMI'].where(datatt['ORIGINAL_BMI'] >= stats.at['ORIGINAL_BMI','IQRlwr']).where(datatt['ORIGINAL_BMI'] <= stats.at['ORIGINAL_BMI','IQRupr'])
    datatt['DIASTOLIC'] = datatt['DIASTOLIC'].where(datatt['DIASTOLIC'] >= stats.at['DIASTOLIC','IQRlwr']).where(datatt['DIASTOLIC'] <= stats.at['DIASTOLIC','IQRupr'])
    datatt['SYSTOLIC'] = datatt['SYSTOLIC'].where(datatt['SYSTOLIC'] >= stats.at['SYSTOLIC','IQRlwr']).where(datatt['SYSTOLIC'] <= stats.at['SYSTOLIC','IQRupr'])

    datatt = datatt.dropna(subset=['WT','SYSTOLIC','DIASTOLIC','ORIGINAL_BMI'])

    datatt.to_pickle('data/'+site+'/p0_'+dataname+'_'+site+'_drop.pkl')
    
    print('Finished p05 '+dataname+' on site '+site, flush = True)                        

In [ ]:
if __name__ == "__main__":
    site='MCRI'
    dataname = 'vital'
    print('Running p05 '+dataname+' on site '+site, flush = True)                    

    datatt = pd.read_pickle('data/'+site+'/p0_'+dataname+'_'+site+'.pkl')

    #stats = datatt['WT'][datatt['WT'].notnull()].agg([np.size, np.mean, np.std, np.median, np.var, np.min, np.max, lambda x: np.percentile(x, q=25), lambda x: np.percentile(x, q=75)]

    datatt2 = pd.melt(datatt, id_vars=[], value_vars=['WT','SYSTOLIC','DIASTOLIC','ORIGINAL_BMI'])

    stats = datatt2[datatt2['value'].notnull()].groupby('variable').agg([np.size, np.mean, np.std, np.median, np.var, np.min, np.max, lambda x: np.percentile(x, q=25), lambda x: np.percentile(x, q=75)])
    stats.columns = ['size','mean','std','median','var','amin','amax','Q1','Q3']
    stats = stats.assign(IQR=lambda x: x.Q3-x.Q1).assign(IQRlwr=lambda x: x['Q1']-1.5*x['IQR']).assign(IQRupr=lambda x: x['Q3']+1.5*x['IQR']).assign(sigmalwr=lambda x: x['mean']-2.7*x['std']).assign(sigmaupr=lambda x: x['mean']+2.7*x['std'])

    datatt['WT'] = datatt['WT'].where(datatt['WT'] >= stats.at['WT','IQRlwr']).where(datatt['WT'] <= stats.at['WT','IQRupr'])
    datatt['ORIGINAL_BMI'] = datatt['ORIGINAL_BMI'].where(datatt['ORIGINAL_BMI'] >= stats.at['ORIGINAL_BMI','IQRlwr']).where(datatt['ORIGINAL_BMI'] <= stats.at['ORIGINAL_BMI','IQRupr'])
    datatt['DIASTOLIC'] = datatt['DIASTOLIC'].where(datatt['DIASTOLIC'] >= stats.at['DIASTOLIC','IQRlwr']).where(datatt['DIASTOLIC'] <= stats.at['DIASTOLIC','IQRupr'])
    datatt['SYSTOLIC'] = datatt['SYSTOLIC'].where(datatt['SYSTOLIC'] >= stats.at['SYSTOLIC','IQRlwr']).where(datatt['SYSTOLIC'] <= stats.at['SYSTOLIC','IQRupr'])

    datatt = datatt.dropna(subset=['WT','SYSTOLIC','DIASTOLIC','ORIGINAL_BMI'])

    datatt.to_pickle('data/'+site+'/p0_'+dataname+'_'+site+'_drop.pkl')
    
    print('Finished p05 '+dataname+' on site '+site, flush = True) 
    
    

In [ ]:
#stats

In [ ]:
#datatt2[datatt2['variable']=='SYSTOLIC'][datatt2['value']>180].value_counts()